# Implied Volatility

#### About the author

This notebook was adapted from this [post](http://blog.nag.com/2013/10/implied-volatility-using-pythons-pandas.html). The original author is Brian Spector from NAG.

### Introduction

In financial analysis, the famous [Black--Scholes formula](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) prices an option as a function of six variables: Underlying Price, Interest Rate, Dividends, Strike Price, Time-to-Expiration, and Volatility. For a given option contract, we can observe the Underlying Price, Interest Rate, and Dividend Rate. In addition, the options contract specifies the Strike Price and Time-to-Expiration.

The one variable we have to tweak is thus the volatility. Let us ask the question: What Volatility is such that the Black--Scholes equation price equals the market price?

$$
F(\text{Volatility}^*) = \text{Market Option Price},
$$

where $\text{Volatility}^*$ is the volatility **implied** by the market price or the [implied volatility](https://en.wikipedia.org/wiki/Implied_volatility). $F$ is a continuous function on $[0, 1]$ so we can use a root finder to get value(s) of $\text{Volatility}^*$ which solve(s) the above equation.

### Using NAG and Python libraries

We can use a few different NAG functions; [nag_zero_cont_func_brent](http://www.nag.com/numeric/CL/nagdoc_cl23/html/C05/c05ayc.html) finds the root using Brent's Algorithm, [nag_bsm_price](http://www.nag.com/numeric/CL/nagdoc_cl23/html/S/s30aac.html) calculates the theoretical option price, [nag_2d_cheb_fit_lines](http://www.nag.com/numeric/CL/nagdoc_cl23/html/E02/e02cac.html) performs a least squares Chebyshev fit to the volatility surface, and [nag_2d_cheb_eval](http://www.nag.com/numeric/CL/nagdoc_cl23/html/E02/e02cbc.html) evaluates the surface at intermediate points.

We will use the popular packages Python offers for numerical computing and data analysis: `numpy`, `scipy`, and `pandas`. The latter has fast and efficient data analysis tools to store and process large amounts of data. In addition, `pandas` comes with `numpy` and `ctypes` built into it, which allows easy integration with NAG's `nag4py` package. Visualizations will be made with `plotly`.

The code below runs on data from the Chicago Board of Options Exchange (CBOE) website. The CBOE provides options data in downloadable format [here](http://www.cboe.com/delayedquote/QuoteTableDownload.aspx). Make sure you download data during CBOE trading hours, so your graphs are not trivial.

Please get the `implied_volatility_utils` module (`implied_volatility_utils.py` file) from [here](http://www.nag.co.uk/blog_files/nag_imp_vol.zip).

In [1]:
# Load one of its functions to make sure you have it in the current working directory.
from implied_volatility_utils import generate_data

## Parsing the data

In [2]:
import pandas as pd

Take a look at the `QuoteData.dat` file; it starts with 2 lines of metadata, and looks like a CSV file.

In [3]:
quotedata = pd.read_csv('QuoteData.dat', header=2)

In [4]:
quotedata.head()

,Calls,Last Sale,Net,Bid,Ask,Vol,Open Int,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Vol.1,Open Int.1,Unnamed: 14
0,15 Jun 65.00 (AAPL1519F65),62.65,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65),0.01,0,0.0,0.01,0,236,NaN
1,15 Jun 65.00 (AAPL1519F65-4),0.00,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-4),0.01,0,0.0,0.01,0,236,NaN
2,15 Jun 65.00 (AAPL1519F65-8),62.55,0,62.00,62.25,0,206,15 Jun 65.00 (AAPL1519R65-8),0.02,0,0.0,0.01,0,236,NaN
3,15 Jun 65.00 (AAPL1519F65-A),0.00,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-A),0.00,0,0.0,0.01,0,236,NaN
4,15 Jun 65.00 (AAPL1519F65-B),0.00,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-B),0.00,0,0.0,0.01,0,236,NaN


In [5]:
quotedata.describe()

,Last Sale,Net,Vol,Open Int,Last Sale.1,Net.1,Vol.1,Open Int.1,Unnamed: 14
count,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,0
mean,13.523344,-0.063621,39.737259,7779.500142,6.033988,0.020449,32.916195,5743.854332,NaN
std,20.289055,0.235779,421.766855,19202.093967,12.257830,0.167572,379.348601,11874.132766,NaN
min,0.000000,-5.310000,0.000000,0.000000,0.000000,-1.950000,0.000000,0.000000,NaN
25%,0.020000,0.000000,0.000000,165.000000,0.010000,0.000000,0.000000,105.000000,NaN
50%,2.900000,0.000000,0.000000,1014.000000,0.530000,0.000000,0.000000,1224.000000,NaN
75%,18.962500,0.000000,0.000000,5759.000000,5.350000,0.000000,0.000000,5428.000000,NaN
max,97.100000,3.300000,22162.000000,174284.000000,72.000000,4.560000,20699.000000,125734.000000,NaN


In [6]:
# Some filtering, clean-up.
quotedata = quotedata.fillna(0.0)
quotedata = quotedata[(quotedata['Last Sale'] > 0) | (quotedata['Last Sale.1'] > 0)]
quotedata = quotedata[quotedata['Calls'] > 0]

In [7]:
quotedata.describe()

,Last Sale,Net,Vol,Open Int,Last Sale.1,Net.1,Vol.1,Open Int.1,Unnamed: 14
count,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334
mean,15.081923,-0.070954,44.317019,8648.153300,6.729411,0.022805,36.709820,6380.140354,0
std,20.870691,0.247951,445.184473,20091.901379,12.762977,0.176814,400.442175,12380.346472,0
min,0.000000,-5.310000,0.000000,0.000000,0.000000,-1.950000,0.000000,0.000000,0
25%,0.150000,0.000000,0.000000,273.000000,0.040000,0.000000,0.000000,269.000000,0
50%,4.825000,0.000000,0.000000,1373.000000,0.920000,0.000000,0.000000,1598.000000,0
75%,22.957500,0.000000,0.000000,7013.000000,6.400000,0.000000,0.000000,6368.000000,0
max,97.100000,3.300000,22162.000000,174284.000000,72.000000,4.560000,20699.000000,125734.000000,0


Dimensions can be accessed in two possible ways:

In [8]:
quotedata.Calls  # or quotedata['Calls']

0       15 Jun 65.00 (AAPL1519F65)
1     15 Jun 65.00 (AAPL1519F65-4)
2     15 Jun 65.00 (AAPL1519F65-8)
5     15 Jun 65.00 (AAPL1519F65-E)
6     15 Jun 65.00 (AAPL1519F65-I)
7     15 Jun 65.00 (AAPL1519F65-J)
8     15 Jun 65.00 (AAPL1519F65-O)
9     15 Jun 65.00 (AAPL1519F65-P)
11    15 Jun 65.00 (AAPL1519F65-X)
12    15 Jun 65.00 (AAPL1519F65-Y)
13      15 Jun 70.00 (AAPL1519F70)
14    15 Jun 70.00 (AAPL1519F70-4)
15    15 Jun 70.00 (AAPL1519F70-8)
17    15 Jun 70.00 (AAPL1519F70-B)
18    15 Jun 70.00 (AAPL1519F70-E)
...
7049    17 Jan 190.00 (AAPL1720A190-X)
7050    17 Jan 190.00 (AAPL1720A190-Y)
7051      17 Jan 195.00 (AAPL1720A195)
7052    17 Jan 195.00 (AAPL1720A195-4)
7053    17 Jan 195.00 (AAPL1720A195-8)
7054    17 Jan 195.00 (AAPL1720A195-A)
7055    17 Jan 195.00 (AAPL1720A195-B)
7056    17 Jan 195.00 (AAPL1720A195-E)
7057    17 Jan 195.00 (AAPL1720A195-I)
7058    17 Jan 195.00 (AAPL1720A195-J)
7059    17 Jan 195.00 (AAPL1720A195-O)
7060    17 Jan 195.00 (AAPL1720A195-P)
706

Let us read the metadata now (we could have started with this).

In [9]:
qd = open('QuoteData.dat', 'r')

In [10]:
qd_head = []
qd_head.append(qd.readline())
qd_head.append(qd.readline())
qd.close()

In [11]:
first_line = qd_head[0].split(',')

In [12]:
second_line = qd_head[1].split()

In [13]:
underlyingprice = float(first_line[1])

In [14]:
# Set to hold expiration dates
cumulative_month = {'Jan': 31, 'Feb': 57, 'Mar': 90,
                    'Apr': 120, 'May': 151, 'Jun': 181,
                    'Jul': 212, 'Aug': 243, 'Sep': 273,
                    'Oct': 304, 'Nov': 334, 'Dec': 365}

month, day = second_line[:2]
today = cumulative_month[month] + int(day) - 30
current_year = int(second_line[2])

In [15]:
# Get the Options Expiration Date
def get_expiration(x):
    monthday = x.split()
    adate = monthday[0] + ' ' + monthday[1]
    return (int(monthday[0]) - (current_year % 2000)) * 365 + cumulative_month[monthday[1]]

In [16]:
expiration = quotedata['Calls'].apply(get_expiration)
expiration.name = 'Expiration'

In [17]:
# Get the Strike Prices
def get_strike(x):
    monthday = x.split()
    return float(monthday[2])

In [18]:
strike = quotedata['Calls'].apply(get_strike)
strike.name = 'Strike'

In [19]:
quotedata = quotedata.join(expiration).join(strike)

Take a look at a slice.

In [20]:
quotedata.ix[1000, :]

Calls          15 Jun 121.00 (AAPL1526F121-Y)
Last Sale                               10.82
Net                                         0
Bid                                      6.30
Ask                                      6.40
Vol                                         0
Open Int                                  434
Puts           15 Jun 121.00 (AAPL1526R121-Y)
Last Sale.1                              0.11
Net.1                                       0
Bid.1                                    0.10
Ask.1                                    0.12
Vol.1                                       0
Open Int.1                               3200
Unnamed: 14                                 0
Expiration                                181
Strike                                    121
Name: 1000, dtype: object

In [21]:
# Call generate_data() to get option prices.

## Calculate Implied Volatility of Calls

In [22]:
impvolcall = pd.Series(pd.np.zeros(len(quotedata.index)), index=quotedata.index, name='impvolCall')

Let us denote the time to expiration by T, the underlying price by S, the interest rate by r, and the dividend rate by q.

### SciPy method

In [23]:
from implied_volatility_utils import scipy_calcvol as calcvol

### Nag4Py method

In [24]:
from implied_volatility_utils import (nag4py_package_check,
                                      nag4py_calcvol as calcvol)
nag4py_package_check()